In [ ]:
!pip install unsloth==2025.9.6
!pip install --no-deps trl==0.8.6 xformers accelerate peft bitsandbytes

In [4]:
import unsloth
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-2-2b-it-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.9.6: Fast Gemma2 patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

Unsloth 2025.9.6 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


In [15]:
from datasets import load_dataset
dataset = load_dataset("roneneldan/TinyStories", split="train[:3000]")

In [16]:
dataset

Dataset({
    features: ['text'],
    num_rows: 3000
})

In [17]:
dataset['text'][0]

'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'

In [18]:
def format_creative_writing(examples):
    """Format examples for creative writing task"""
    texts = []

    for i in range(len(examples["text"])):
        story = examples["text"][i]

        # Extract first sentence as prompt, rest as story
        sentences = story.split('. ')
        if len(sentences) > 2:
            prompt = sentences[0] + '.'
            continuation = '. '.join(sentences[1:])

            # Create creative writing format
            messages = [
                {"role": "user", "content": f"Continue this story creatively: {prompt}"},
                {"role": "model", "content": continuation}
            ]

            # Apply Gemma-2 chat template
            text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=False,
            )

            texts.append(text)

    return {"text": texts}

In [19]:
formatted_dataset = dataset.map(
    format_creative_writing,
    batched=True,
    remove_columns=dataset.column_names,
    desc="Formatting for creative writing"
)

Formatting for creative writing:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [20]:
formatted_dataset['text'][0]

'<bos><start_of_turn>user\nContinue this story creatively: One day, a little girl named Lily found a needle in her room.<end_of_turn>\n<start_of_turn>model\nShe knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.<end_of_turn>\n'

In [21]:
formatted_dataset = formatted_dataset.filter(lambda x: x["text"] is not None)

Filter:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [22]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,  # Larger batch for 2B model
    gradient_accumulation_steps=2,  # Effective batch size: 8
    warmup_steps=10,
    max_steps=100,  # Sufficient for creative patterns
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=5,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    seed=3407,
    output_dir="gemma2_creative_writer",
    save_strategy="no",
)

In [24]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    args=training_args,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=False,
)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [25]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,000 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 20,766,720 of 2,635,108,608 (0.79% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abeshith (abeshith-dr-m-g-r-educational-and-research-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,2.776100
10,2.175500
15,1.687100
20,1.545700
25,1.577100
30,1.427000
35,1.376400
40,1.417500
45,1.363700
50,1.382800


TrainOutput(global_step=100, training_loss=1.5027860498428345, metrics={'train_runtime': 384.2938, 'train_samples_per_second': 2.082, 'train_steps_per_second': 0.26, 'total_flos': 3263243410925568.0, 'train_loss': 1.5027860498428345, 'epoch': 0.26666666666666666})

In [26]:
model.save_pretrained("gemma2_story_writer")
tokenizer.save_pretrained("gemma2_story_writer")

('gemma2_story_writer/tokenizer_config.json',
 'gemma2_story_writer/special_tokens_map.json',
 'gemma2_story_writer/chat_template.jinja',
 'gemma2_story_writer/tokenizer.model',
 'gemma2_story_writer/added_tokens.json',
 'gemma2_story_writer/tokenizer.json')

In [ ]:
FastLanguageModel.for_inference(model)

In [28]:
def generate_creative_content(prompt, content_type="story", max_tokens=600):
    """Generate creative content based on prompt and type"""

    prompts_by_type = {
        "story": f"Write a creative and engaging story based on this prompt: {prompt}",
        "dialogue": f"Write realistic dialogue for this scenario: {prompt}",
        "character": f"Create a detailed character description for: {prompt}",
        "scene": f"Write a vivid scene description for: {prompt}",
        "plot": f"Develop a plot outline for: {prompt}",
        "poem": f"Write a creative poem about: {prompt}",
        "continue": f"Continue this story creatively: {prompt}"
    }

    messages = [
        {"role": "user", "content": prompts_by_type.get(content_type, prompts_by_type["story"])}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer([text], return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.8,  # Higher temp for creativity
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,
        )

    # Extract generated tokens and decode
    generated_tokens = outputs[0][len(inputs['input_ids'][0]):]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return response.strip()

In [29]:
story_prompts = [
    "A detective discovers that their reflection has been acting independently",
    "The last bookstore on Earth receives a mysterious customer",
    "A time traveler gets stuck in a ten-minute loop",
    "Two rival chefs must work together to save their neighborhood",
    "A small town where everyone can hear each other's thoughts"
]

print("📚 STORY GENERATION:")
for i, prompt in enumerate(story_prompts, 1):
    print(f"\n✨ Story {i}: {prompt}")
    story = generate_creative_content(prompt, "story", 400)
    print(f"Generated Story: {story}")
    print("-" * 60)

📚 STORY GENERATION:

✨ Story 1: A detective discovers that their reflection has been acting independently
Generated Story: Detective Johnson had solved many cases, but nothing had prepared him for the events of today. He was trying to find a missing piece for his client's jewelry case when he looked into the mirror. To his shock, he saw something very strange. His reflection wasn't just looking back; it was moving around on its own!

Feeling confused, Detective Johnson investigated further. He found out that his reflection could talk, move, and even do some pretty wacky things like pick up objects. He discovered that it was acting all by itself, making decisions without consulting him. 

The situation felt too bizarre to be real. So, Detective Johnson decided to test if his reflection was truly independent or not. He used his knowledge about psychology to create one last experiment. He asked his reflection what was wrong with his world so he wouldn't look at it in a negative way.

His 

In [30]:
character_prompts = [
    "A retired superhero who now works as a librarian",
    "A baker who can taste people's emotions in their bread",
    "A young inventor living in a steampunk Victorian city",
    "A space archaeologist exploring ancient alien ruins",
    "A street artist whose drawings come to life at midnight"
]

print("\n👥 CHARACTER DEVELOPMENT:")
for i, prompt in enumerate(character_prompts, 1):
    print(f"\n🎭 Character {i}: {prompt}")
    character = generate_creative_content(prompt, "character", 300)
    print(f"Character Description: {character}")
    print("-" * 60)


👥 CHARACTER DEVELOPMENT:

🎭 Character 1: A retired superhero who now works as a librarian
Character Description: Imagine this: He's in his apron, reading aloud to kids.

He is an older gentleman with bright eyes and kind smiles. He loves helping people, especially children. His past life was full of adventures, so he can help the young ones understand their world better.
------------------------------------------------------------

🎭 Character 2: A baker who can taste people's emotions in their bread
Character Description: Name: Lily

Lily was an average person... unless she was baking! When it came to her job, she could always tell when someone was happy or sad just by looking at the bread. For example, if you were feeling sad, your bread would taste like sadness. If you were feeling good, your bread would taste like happiness. She could even tell what kind of emotion was inside the bread. It was impressive but also frustrating because she couldn't figure out why some people felt so 

In [31]:
dialogue_scenarios = [
    "Two astronauts discovering they're not alone on a distant planet",
    "A parent explaining to their child why they have to move to a new city",
    "Two old friends meeting after 20 years with a secret between them",
    "A job interview for the world's first time travel tour guide",
    "Two AIs having their first conversation about human emotions"
]

print("\n💬 DIALOGUE WRITING:")
for i, scenario in enumerate(dialogue_scenarios, 1):
    print(f"\n🗣️ Scenario {i}: {scenario}")
    dialogue = generate_creative_content(scenario, "dialogue", 350)
    print(f"Generated Dialogue: {dialogue}")
    print("-" * 60)


💬 DIALOGUE WRITING:

🗣️ Scenario 1: Two astronauts discovering they're not alone on a distant planet
Generated Dialogue: Scene: A lonely room in an astronaut's base.

As the two astronauts return from a mission, Dr. Johnson enters the room and excitedly tells his colleague, Dr. Miller. They both look at the news flash screen showing images of strange objects and shapes.

Dr. Johnson: "Look, Doc! I saw something weird down there."
Dr. Miller: "It's just rocks, Jo-Jo. Don't worry about it." 
Dr. Johnson: "But wait, look again! There are no rocks there!"
Dr. Miller: "Oh my god, you're right! It looks like some sort of shape or object down there. We should investigate that area immediately."
Dr. Johnson: "I agree, let's go see!"

The two astronauts then enter their space suit and fly off to check out the strange shape on the other side of the moon. As they approach it, they notice that it is moving towards them. The two astronauts begin to get nervous when suddenly they hear an engine roa

In [32]:
scene_prompts = [
    "An abandoned amusement park during a thunderstorm",
    "A bustling alien marketplace on a distant moon",
    "The inside of a magical library where books float freely",
    "A coffee shop that exists between dimensions",
    "The last day of school in a post-apocalyptic world"
]

print("\n🎬 SCENE DESCRIPTIONS:")
for i, prompt in enumerate(scene_prompts, 1):
    print(f"\n🌅 Scene {i}: {prompt}")
    scene = generate_creative_content(prompt, "scene", 300)
    print(f"Scene Description: {scene}")
    print("-" * 60)


🎬 SCENE DESCRIPTIONS:

🌅 Scene 1: An abandoned amusement park during a thunderstorm
Scene Description: The sun was gone. The air tasted of rain and the ground, still muddy from the morning, started to show some wet patches. It wasn't a bad day, just overcast and gloomy. But one thing was for sure – it wouldn't be fun to try and ride any of these attractions right now.

Suddenly, something very loud appeared in the distance. People ran and shouted, but nobody seemed to understand why. Then a big flash of white light showed up. That meant one thing - somebody had gotten their hands on the lightning rod! Everyone cheered and jumped for joy. Soon enough, everyone was out of the park, scared by the wild display of lightning. Nobody cared about what made it so special or how they got there. All they knew was that the sky was wild and the weather was really getting worse.
------------------------------------------------------------

🌅 Scene 2: A bustling alien marketplace on a distant moon
S

In [33]:
plot_concepts = [
    "A world where memories can be traded like currency",
    "A detective story set in a city where it never stops raining",
    "A romance between a human and an AI who communicates through music",
    "A heist movie where the thieves steal abstract concepts instead of objects",
    "A coming-of-age story in a generation ship traveling to another star"
]

print("\n📖 PLOT DEVELOPMENT:")
for i, concept in enumerate(plot_concepts, 1):
    print(f"\n📝 Plot {i}: {concept}")
    plot = generate_creative_content(concept, "plot", 400)
    print(f"Plot Outline: {plot}")
    print("-" * 60)


📖 PLOT DEVELOPMENT:

📝 Plot 1: A world where memories can be traded like currency
Plot Outline: ## Plot Outline: Memory Trading

**Logline:** In the future, people trade their treasured memories to make money. However, one man discovers that his memory is worth more than he knows.

**Characters**:
- Jack: An ordinary man who's struggling to make ends meet.
- Sarah: Jack's girlfriend who wants him to sell her memory of winning a lottery.
- The Memory Trader: A mysterious figure who collects and sells unusual memories.

**Plot Points:**

1. **Exposition**: Jack tries to manage life with limited resources. He finds out about the existence of memory trading, where rare and valuable memories are sold for huge sums. Jack decides to see if he can make some extra money by selling his memories or keeping them safe. 

2. **Rising Action**: Jack realizes his everyday memories are not very interesting to other traders, so he needs to find something special to offer. One day, he decides to find Sa

In [34]:
story_beginnings = [
    "She found the letter under her pillow, written in her own handwriting, but she had no memory of writing it.",
    "The museum was supposed to be closed, but all the lights were on and there was music playing inside.",
    "Every morning at exactly 7:23 AM, the same stranger waved at him from the train platform.",
    "The antique shop appeared overnight, and its owner claimed to sell memories instead of objects.",
    "When she opened her laptop, there was a message from herself: 'Don't trust anyone, including me.'"
]

print("\n🔄 STORY CONTINUATION:")
for i, beginning in enumerate(story_beginnings, 1):
    print(f"\n📖 Beginning {i}: {beginning}")
    continuation = generate_creative_content(beginning, "continue", 350)
    print(f"Story Continuation: {continuation}")
    print("-" * 60)


🔄 STORY CONTINUATION:

📖 Beginning 1: She found the letter under her pillow, written in her own handwriting, but she had no memory of writing it.
Story Continuation: She was confused and worried so she decided to take a look at when the letter was written. It turned out the date on the paper was from two days ago.

She looked around and saw that there were only flowers left in the vase from yesterday. They didn't smell like anything bad, but they still made her think.

Suddenly, she remembered something. When she had been feeling sleepy and tired, she had seen a few letters lying in the living room. So she picked them up and took a closer look. The script was all over the place, making her even more confused.

But then she noticed one thing. Somehow the letters had been folded into each other. She carefully unfolded them and realized how much work it must have taken for her to write such messy lines. She now knew where she had found the letter and why it smelled so sweet.
------------

In [35]:
poetry_themes = [
    "The feeling of watching sunrise from space",
    "A conversation between the ocean and the moon",
    "The last tree in a concrete world",
    "Digital love in an analog heart",
    "The sound of silence in a busy city"
]

print("\n🎭 POETRY GENERATION:")
for i, theme in enumerate(poetry_themes, 1):
    print(f"\n🌟 Theme {i}: {theme}")
    poem = generate_creative_content(theme, "poem", 200)
    print(f"Generated Poem: {poem}")
    print("-" * 60)


🎭 POETRY GENERATION:

🌟 Theme 1: The feeling of watching sunrise from space
Generated Poem: I was in the air, floating around in a big white suit. I saw a bright light in the sky and it gave me chills. Then, I started to feel my heart beating fast. 

Suddenly, I realized what was happening! I was witnessing the beginning of a new day. The morning sun had risen above the horizon, making everything bright and beautiful. It was truly special to be here and experience such a magical moment. 

As I looked out at this wondrous sight, I couldn't help but smile. I felt happy and peaceful all over. It was an amazing feeling. From that moment on, I knew that I wanted to explore more of the world and see even more of its wonders.
------------------------------------------------------------

🌟 Theme 2: A conversation between the ocean and the moon
Generated Poem: The ocean whispered, "It's so calm here tonight."

The moon smiled and replied, "I can see your waves from up above."

"Oh, it's so pea